In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xg
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf

In [2]:
diamonds_train = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/Proyecto5/Diamond-Price-Prediction/train.csv')

In [42]:
diamonds_test = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/Proyecto5/Diamond-Price-Prediction/test.csv')

In [4]:
diamonds_train.shape

(40455, 11)

In [5]:
diamonds_train.head(3)

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,0.30,Premium,D,SI2,62.4,58.0,4.31,4.28,2.68,6.353
1,1,1.01,Ideal,E,VVS2,62.7,56.0,6.42,6.46,4.04,9.183
2,2,0.72,Ideal,F,VS2,61.8,59.0,5.71,5.74,3.54,7.983


#### CATEGORICAL FEATURES

In [6]:
diamonds_train.cut.value_counts()

Ideal        16230
Premium      10338
Very Good     9003
Good          3682
Fair          1202
Name: cut, dtype: int64

In [7]:
dic_cut = {'Ideal': '1', 'Premium': '2', 'Very Good': '3', 'Good': '4', 'Fair': '5'}

In [8]:
diamonds_train.cut = diamonds_train.cut.map(dic_cut)

#### COLOR IS MORE IMPORTANT THAN CLARITY (ojo)

In [9]:
diamonds_train.color.value_counts()

G    8410
E    7368
F    7182
H    6203
D    5100
I    4070
J    2122
Name: color, dtype: int64

In [10]:
#THE COLOR SCALE STARTS AT D, WICH IS NOT TOP QUALITY

In [11]:
dic_color = {'D': '1', 'E': '2', 'F': '3', 'G': '4', 'H': '5', 'I': '6', 'J': '7'}

In [12]:
diamonds_train.color = diamonds_train.color.map(dic_color)

In [13]:
diamonds_train.color.value_counts()

4    8410
2    7368
3    7182
5    6203
1    5100
6    4070
7    2122
Name: color, dtype: int64

In [14]:
diamonds_train.clarity.unique()

array(['SI2', 'VVS2', 'VS2', 'VS1', 'SI1', 'VVS1', 'IF', 'I1'],
      dtype=object)

In [15]:
dic_clarity = {'I1': '1', 'SI2': '2', 'SI1': '3', 'VS2': '4', 'VS1': '5', 'VVS2': '6', 'VVS1': '7', 'IF': '8'}

In [16]:
diamonds_train.clarity = diamonds_train.clarity.map(dic_clarity)

In [17]:
X = diamonds_train.drop('price', axis=1)

In [18]:
y = diamonds_train.price

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#### RANDOM FOREST

In [ ]:
#rs = RandomizedSearchCV(
    estimator=RandomForestRegressor(),
    param_distributions={
        'bootstrap': [True, False],
        'max_depth': [10, 11, 12, 13, 14, 15, 16, None],
        'max_features': [5, 6, 7, 8, 9],
        'min_samples_split': [15, 30, 50],
        'n_estimators': [50, 100, 150]
    },
    n_iter=100,
    cv=10,
    verbose=2,
    scoring="neg_root_mean_squared_error",
    return_train_score=True
)

In [ ]:
%%time
#rs.fit(X_train, y_train)

In [ ]:
best_ranf = rs.best_estimator_
best_ranf

#### RANDOM FOREST (BASADO EN EL random SEARCH ANTERIOR)

In [ ]:
random_forest2 = RandomForestRegressor(bootstrap=False, max_depth=16, max_features=5, min_samples_split=15, n_estimators=150)

In [ ]:
random_forest2.fit(X_train, y_train)

In [ ]:
y_test_pred = random_forest2.predict(X_test)

In [ ]:
(mean_squared_error(y_test, y_test_pred)) ** 0.5

#### Random Forest con XGBOOST 

In [21]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()

In [22]:
X_train['cut'] = lbl.fit_transform(X_train['cut'].astype(str))
X_train['color'] = lbl.fit_transform(X_train['color'].astype(str))
X_train['clarity'] = lbl.fit_transform(X_train['clarity'].astype(str))

<ipython-input-22-41c8d66383be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['cut'] = lbl.fit_transform(X_train['cut'].astype(str))
<ipython-input-22-41c8d66383be>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['color'] = lbl.fit_transform(X_train['color'].astype(str))
<ipython-input-22-41c8d66383be>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [23]:
X_test['cut'] = lbl.fit_transform(X_test['cut'].astype(str))
X_test['color'] = lbl.fit_transform(X_test['color'].astype(str))
X_test['clarity'] = lbl.fit_transform(X_test['clarity'].astype(str))

<ipython-input-23-6b48c5ad10ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['cut'] = lbl.fit_transform(X_test['cut'].astype(str))
<ipython-input-23-6b48c5ad10ca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['color'] = lbl.fit_transform(X_test['color'].astype(str))
<ipython-input-23-6b48c5ad10ca>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [24]:
train_dmatrix = xg.DMatrix(data=X_train, label=y_train, enable_categorical=True)

In [25]:
test_dmatrix = xg.DMatrix(data =X_test, label=y_test, enable_categorical=True)

In [27]:
params = {'booster':'gbtree', 
  'objective':'reg:linear', 
  'colsample_bynode': 0.8,
  'learning_rate': 0.3,
  'max_depth': 15,
  'num_parallel_tree': 50,
  'subsample': 0.8,
  'tree_method': 'auto',
  'verbosity': 3}

In [28]:
xgb_reg = xg.train(params = params, dtrain = train_dmatrix, num_boost_round = 50)

[11:58:18] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[11:58:18] DEBUG: ../src/gbm/gbtree.cc:155: Using tree method: 2
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=5
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=5
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=6
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=5
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=6
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=6
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=6
[11:58:19] INFO: ../src/t

[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=6
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=7
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=7
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=6
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=6
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=6
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=7
[11:58:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned no

[11:58:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 90 extra nodes, 0 pruned nodes, max_depth=9
[11:58:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 92 extra nodes, 0 pruned nodes, max_depth=7
[11:58:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 100 extra nodes, 0 pruned nodes, max_depth=9
[11:58:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 0 pruned nodes, max_depth=7
[11:58:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 92 extra nodes, 0 pruned nodes, max_depth=8
[11:58:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 94 extra nodes, 0 pruned nodes, max_depth=8
[11:58:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 90 extra nodes, 0 pruned nodes, max_depth=8
[11:58:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 0 pruned nodes, max_depth=8
[11:58:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 76 extra nodes, 0 pruned n

[11:58:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 174 extra nodes, 0 pruned nodes, max_depth=11
[11:58:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 206 extra nodes, 0 pruned nodes, max_depth=11
[11:58:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 208 extra nodes, 0 pruned nodes, max_depth=14
[11:58:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 202 extra nodes, 0 pruned nodes, max_depth=14
[11:58:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 178 extra nodes, 0 pruned nodes, max_depth=11
[11:58:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 186 extra nodes, 0 pruned nodes, max_depth=11
[11:58:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 198 extra nodes, 0 pruned nodes, max_depth=12
[11:58:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 194 extra nodes, 0 pruned nodes, max_depth=13
[11:58:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 198 extra n

[11:58:24] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 560 extra nodes, 0 pruned nodes, max_depth=15
[11:58:24] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 646 extra nodes, 0 pruned nodes, max_depth=15
[11:58:24] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 688 extra nodes, 0 pruned nodes, max_depth=15
[11:58:24] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 652 extra nodes, 0 pruned nodes, max_depth=15
[11:58:24] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 636 extra nodes, 0 pruned nodes, max_depth=15
[11:58:24] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 668 extra nodes, 0 pruned nodes, max_depth=15
[11:58:24] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 550 extra nodes, 0 pruned nodes, max_depth=15
[11:58:24] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 552 extra nodes, 0 pruned nodes, max_depth=15
[11:58:24] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 620 extra n

...

[11:59:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3988 extra nodes, 0 pruned nodes, max_depth=15
[11:59:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2714 extra nodes, 0 pruned nodes, max_depth=15
[11:59:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2636 extra nodes, 0 pruned nodes, max_depth=15
[11:59:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5334 extra nodes, 0 pruned nodes, max_depth=15
[11:59:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3992 extra nodes, 0 pruned nodes, max_depth=15
[11:59:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3684 extra nodes, 0 pruned nodes, max_depth=15
[11:59:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3772 extra nodes, 0 pruned nodes, max_depth=15
[11:59:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1588 extra nodes, 0 pruned nodes, max_depth=15
[11:59:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 223

[11:59:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4416 extra nodes, 0 pruned nodes, max_depth=15
[11:59:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4548 extra nodes, 0 pruned nodes, max_depth=15
[11:59:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6222 extra nodes, 0 pruned nodes, max_depth=15
[11:59:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2608 extra nodes, 0 pruned nodes, max_depth=15
[11:59:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3852 extra nodes, 0 pruned nodes, max_depth=15
[11:59:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5514 extra nodes, 0 pruned nodes, max_depth=15
[11:59:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2288 extra nodes, 0 pruned nodes, max_depth=15
[11:59:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3124 extra nodes, 0 pruned nodes, max_depth=15
[11:59:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 411

[11:59:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3492 extra nodes, 0 pruned nodes, max_depth=15
[11:59:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5198 extra nodes, 0 pruned nodes, max_depth=15
[11:59:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2962 extra nodes, 0 pruned nodes, max_depth=15
[11:59:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4846 extra nodes, 0 pruned nodes, max_depth=15
[11:59:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4550 extra nodes, 0 pruned nodes, max_depth=15
[11:59:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4398 extra nodes, 0 pruned nodes, max_depth=15
[11:59:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1180 extra nodes, 0 pruned nodes, max_depth=15
[11:59:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5854 extra nodes, 0 pruned nodes, max_depth=15
[11:59:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 518

[11:59:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2020 extra nodes, 0 pruned nodes, max_depth=15
[11:59:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2118 extra nodes, 0 pruned nodes, max_depth=15
[11:59:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2840 extra nodes, 0 pruned nodes, max_depth=15
[11:59:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3298 extra nodes, 0 pruned nodes, max_depth=15
[11:59:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4944 extra nodes, 0 pruned nodes, max_depth=15
[11:59:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1916 extra nodes, 0 pruned nodes, max_depth=15
[11:59:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2016 extra nodes, 0 pruned nodes, max_depth=15
[11:59:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6422 extra nodes, 0 pruned nodes, max_depth=15
[11:59:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 245

[11:59:43] ======== Monitor: Learner ========
[11:59:43] Configure: 0s, 1 calls @ 0us

[11:59:43] EvalOneIter: 0.003126s, 50 calls @ 3126us

[11:59:43] GetGradient: 0.018134s, 50 calls @ 18134us

[11:59:43] PredictRaw: 0.925842s, 50 calls @ 925842us

[11:59:43] UpdateOneIter: 64.2331s, 50 calls @ 64233100us

[11:59:43] ======== Monitor: GBTree ========
[11:59:43] BoostNewTrees: 63.655s, 50 calls @ 63655015us

[11:59:43] CommitModel: 9.3e-05s, 50 calls @ 93us

[11:59:43] ======== Monitor: TreePruner ========
[11:59:43] PrunerUpdate: 0.399975s, 50 calls @ 399975us



In [29]:
y_test_pred = xgb_reg.predict(test_dmatrix)

In [30]:
(mean_squared_error(y_test, y_test_pred)) ** 0.5

0.0852532752097859

#### RANDOMIZED SEARCH CON XGBOOST

In [58]:
params = { 'max_depth': [14, 15, 16],
           'learning_rate': [0.01, 0.1, 0.2, 0.3, 1],
           'subsample': np.arange(0.5, 0.8, 1),
           'colsample_bytree': np.arange(0.5, 0.8, 1.0),
           'colsample_bylevel': np.arange(0.5, 0.8, 1.0),
           'colsample_bynode': np.arange(0.5, 0.8, 1.0),
           'n_estimators': [25, 50, 100]}

In [59]:
xgbreg = xg.XGBRegressor()

In [61]:
xgb_ransea = RandomizedSearchCV(estimator=xgbreg,
                         param_distributions=params,
                         scoring='neg_root_mean_squared_error',
                         n_iter=50,
                         verbose=3)

In [62]:
xgb_ransea.fit(X_train, y_train)

/home/julian/miniconda3/envs/ironcon/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 45 is smaller than n_iter=50. Running 45 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[13:50:45] DEBUG: ../src/gbm/gbtree.cc:155: Using tree method: 2
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=3
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 p

[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4 extra nodes, 0 pruned nodes, max_depth=2
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=4
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=3
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 10 extra nodes, 0 pruned nodes, max_depth=3
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=5
[13:50:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, m

[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2 extra nodes, 0 pruned nodes, max_depth=1
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4 extra nodes, 0 pruned nodes, max_depth=2
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4 extra nodes, 0 pruned nodes, max

[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=5
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4 extra nodes, 0 pruned nodes, max_depth=2
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=5
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 16 extra nodes, 0 pruned nodes, max_depth=4
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 16 extra nodes, 0 pruned nodes, max_depth=5
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=3
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[13:50:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 16 extra nodes, 0 pruned nodes

...

[13:52:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3048 extra nodes, 0 pruned nodes, max_depth=16
[13:52:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2638 extra nodes, 0 pruned nodes, max_depth=16
[13:52:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 474 extra nodes, 0 pruned nodes, max_depth=15
[13:52:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1966 extra nodes, 0 pruned nodes, max_depth=16
[13:52:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3872 extra nodes, 0 pruned nodes, max_depth=16
[13:52:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2710 extra nodes, 0 pruned nodes, max_depth=16
[13:52:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2664 extra nodes, 0 pruned nodes, max_depth=16
[13:52:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2744 extra nodes, 0 pruned nodes, max_depth=16
[13:52:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 332 

[13:52:54] ======== Monitor: Learner ========
[13:52:54] Configure: 0.000622s, 1 calls @ 622us

[13:52:54] EvalOneIter: 0.000339s, 100 calls @ 339us

[13:52:54] GetGradient: 0.013418s, 100 calls @ 13418us

[13:52:54] PredictRaw: 0.047255s, 100 calls @ 47255us

[13:52:54] UpdateOneIter: 0.924916s, 100 calls @ 924916us

[13:52:54] ======== Monitor: GBTree ========
[13:52:54] BoostNewTrees: 0.8623s, 100 calls @ 862300us

[13:52:54] CommitModel: 6.2e-05s, 100 calls @ 62us

[13:52:54] ======== Monitor: TreePruner ========
[13:52:54] PrunerUpdate: 0.026362s, 100 calls @ 26362us

[CV 4/5] END colsample_bylevel=0.5, colsample_bynode=0.5, colsample_bytree=0.5, learning_rate=1, max_depth=16, n_estimators=100, subsample=0.5;, score=-0.346 total time=   1.5s
[13:52:54] ======== Monitor: Learner ========
[13:52:54] Configure: 0.0004s, 1 calls @ 400us

[13:52:54] ======== Monitor: GBTree ========
[13:52:54] ======== Monitor: TreePruner ========
[13:52:54] DEBUG: ../src/gbm/gbtree.cc:155: Using tree 

[13:52:55] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5290 extra nodes, 0 pruned nodes, max_depth=16
[13:52:55] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4028 extra nodes, 0 pruned nodes, max_depth=16
[13:52:55] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1792 extra nodes, 0 pruned nodes, max_depth=16
[13:52:55] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3992 extra nodes, 0 pruned nodes, max_depth=16
[13:52:55] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2980 extra nodes, 0 pruned nodes, max_depth=16
[13:52:55] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2882 extra nodes, 0 pruned nodes, max_depth=16
[13:52:55] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 94 extra nodes, 0 pruned nodes, max_depth=7
[13:52:55] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1326 extra nodes, 0 pruned nodes, max_depth=16
[13:52:55] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2934 e

[13:52:56] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1244 extra nodes, 0 pruned nodes, max_depth=15
[13:52:56] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1900 extra nodes, 0 pruned nodes, max_depth=15
[13:52:56] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 746 extra nodes, 0 pruned nodes, max_depth=15
[13:52:56] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 952 extra nodes, 0 pruned nodes, max_depth=15
[13:52:56] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2364 extra nodes, 0 pruned nodes, max_depth=15
[13:52:56] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2324 extra nodes, 0 pruned nodes, max_depth=15
[13:52:56] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2940 extra nodes, 0 pruned nodes, max_depth=15
[13:52:56] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2374 extra nodes, 0 pruned nodes, max_depth=15
[13:52:56] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 344 e

RandomizedSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs...
                                          scale_pos_weight=None, subsample=None,
                                          tree_method=None,
                                          validate_parameters=None,
 

In [63]:
print("Best parameters:", xgb_ransea.best_params_)
print("Lowest RMSE: ", (-xgb_ransea.best_score_)**(1/2.0))

Best parameters: {'subsample': 0.5, 'n_estimators': 100, 'max_depth': 15, 'learning_rate': 0.2, 'colsample_bytree': 0.5, 'colsample_bynode': 0.5, 'colsample_bylevel': 0.5}
Lowest RMSE:  0.3254118919273502


#### AHORA, CONSTRUIMOS DE NUEVO EL RANDOM FOREST CON XGBOOST, PERO CON LOS HYPERPARAMETROS OBTENIDOS EN EL RANDOMIZED SEARCH

In [75]:
params = {'booster':'gbtree', 
  'objective':'reg:squarederror', 
  'colsample_bynode': 0.5,
  'learning_rate': 0.2,
  'max_depth': 15,
  'num_parallel_tree': 100,
  'subsample': 0.6,
  'tree_method': 'auto',
  'verbosity': 3}

In [ ]:
xgb_reg = xg.train(params = params, dtrain = train_dmatrix, num_boost_round = 50)

[14:14:19] DEBUG: ../src/gbm/gbtree.cc:155: Using tree method: 2
[14:14:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 16 extra nodes, 0 pruned nodes, max_depth=4
[14:14:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=5
[14:14:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=6
[14:14:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=4
[14:14:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=5
[14:14:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=5
[14:14:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=4
[14:14:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 16 extra nodes, 0 pruned nodes, max_depth=4
[14:14:19] INFO: ../src/tree/up

[14:14:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=4
[14:14:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=6
[14:14:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=4
[14:14:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 16 extra nodes, 0 pruned nodes, max_depth=5
[14:14:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=5
[14:14:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=5
[14:14:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=5
[14:14:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6
[14:14:20] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned no

[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=6
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=6
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=5
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=5
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=6
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 24 extra nodes, 0 pruned nodes, max_depth=5
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=6
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned no

[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=6
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=5
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=5
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=6
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=6
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=5
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=6
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 36 extra nodes, 0 pruned nodes, max_depth=6
[14:14:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned no

[14:14:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=7
[14:14:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 56 extra nodes, 0 pruned nodes, max_depth=6
[14:14:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=6
[14:14:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=7
[14:14:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=6
[14:14:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[14:14:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=5
[14:14:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=6
[14:14:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 50 extra nodes, 0 pruned no

...

[14:15:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3866 extra nodes, 0 pruned nodes, max_depth=15
[14:15:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 764 extra nodes, 0 pruned nodes, max_depth=15
[14:15:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3144 extra nodes, 0 pruned nodes, max_depth=15
[14:15:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4738 extra nodes, 0 pruned nodes, max_depth=15
[14:15:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2488 extra nodes, 0 pruned nodes, max_depth=15
[14:15:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3450 extra nodes, 0 pruned nodes, max_depth=15
[14:15:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2100 extra nodes, 0 pruned nodes, max_depth=15
[14:15:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3896 extra nodes, 0 pruned nodes, max_depth=15
[14:15:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3252

[14:15:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2976 extra nodes, 0 pruned nodes, max_depth=15
[14:15:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2752 extra nodes, 0 pruned nodes, max_depth=15
[14:15:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3468 extra nodes, 0 pruned nodes, max_depth=15
[14:15:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3882 extra nodes, 0 pruned nodes, max_depth=15
[14:15:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3428 extra nodes, 0 pruned nodes, max_depth=15
[14:15:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3140 extra nodes, 0 pruned nodes, max_depth=15
[14:15:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2166 extra nodes, 0 pruned nodes, max_depth=15
[14:15:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3156 extra nodes, 0 pruned nodes, max_depth=15
[14:15:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 378

[14:15:43] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2204 extra nodes, 0 pruned nodes, max_depth=15
[14:15:43] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2330 extra nodes, 0 pruned nodes, max_depth=15
[14:15:43] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3770 extra nodes, 0 pruned nodes, max_depth=15
[14:15:43] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3414 extra nodes, 0 pruned nodes, max_depth=15
[14:15:43] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3040 extra nodes, 0 pruned nodes, max_depth=15
[14:15:43] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3516 extra nodes, 0 pruned nodes, max_depth=15
[14:15:43] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4314 extra nodes, 0 pruned nodes, max_depth=15
[14:15:43] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2664 extra nodes, 0 pruned nodes, max_depth=15
[14:15:43] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 342

[14:15:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4042 extra nodes, 0 pruned nodes, max_depth=15
[14:15:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2484 extra nodes, 0 pruned nodes, max_depth=15
[14:15:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3110 extra nodes, 0 pruned nodes, max_depth=15
[14:15:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4234 extra nodes, 0 pruned nodes, max_depth=15
[14:15:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2006 extra nodes, 0 pruned nodes, max_depth=15
[14:15:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3898 extra nodes, 0 pruned nodes, max_depth=15
[14:15:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2306 extra nodes, 0 pruned nodes, max_depth=15
[14:15:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3128 extra nodes, 0 pruned nodes, max_depth=15
[14:15:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 288

[14:15:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4516 extra nodes, 0 pruned nodes, max_depth=15
[14:15:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3860 extra nodes, 0 pruned nodes, max_depth=15
[14:15:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2518 extra nodes, 0 pruned nodes, max_depth=15
[14:15:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3590 extra nodes, 0 pruned nodes, max_depth=15
[14:15:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5222 extra nodes, 0 pruned nodes, max_depth=15
[14:15:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4788 extra nodes, 0 pruned nodes, max_depth=15
[14:15:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3242 extra nodes, 0 pruned nodes, max_depth=15
[14:15:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3060 extra nodes, 0 pruned nodes, max_depth=15
[14:15:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 383

In [77]:
y_test_pred = xgb_reg.predict(test_dmatrix)

In [78]:
(mean_squared_error(y_test, y_test_pred)) ** 0.5

0.09416332747728197

#### ENTRENAR SOBRE TODO EL TEST!

In [32]:
X['cut'] = lbl.fit_transform(X['cut'].astype(str))
X['color'] = lbl.fit_transform(X['color'].astype(str))
X['clarity'] = lbl.fit_transform(X['clarity'].astype(str))

In [33]:
fulltrain_dmatrix = xg.DMatrix(data=X, label=y, enable_categorical=True)

In [34]:
xgb_reg = xg.train(params = params, dtrain = fulltrain_dmatrix, num_boost_round = 50)

[12:02:57] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[12:02:57] DEBUG: ../src/gbm/gbtree.cc:155: Using tree method: 2
[12:02:57] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=7
[12:02:57] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=7
[12:02:57] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=6
[12:02:57] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=6
[12:02:57] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=7
[12:02:57] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=6
[12:02:57] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=6
[12:02:57] INFO: ../src/t

[12:02:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=7
[12:02:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=7
[12:02:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=7
[12:02:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=6
[12:02:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=8
[12:02:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=6
[12:02:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=7
[12:02:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 52 extra nodes, 0 pruned nodes, max_depth=7
[12:02:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 50 extra nodes, 0 pruned no

[12:02:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 106 extra nodes, 0 pruned nodes, max_depth=8
[12:02:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 102 extra nodes, 0 pruned nodes, max_depth=9
[12:02:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 0 pruned nodes, max_depth=8
[12:02:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 100 extra nodes, 0 pruned nodes, max_depth=8
[12:02:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 100 extra nodes, 0 pruned nodes, max_depth=9
[12:02:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 110 extra nodes, 0 pruned nodes, max_depth=9
[12:02:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 108 extra nodes, 0 pruned nodes, max_depth=9
[12:02:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 92 extra nodes, 0 pruned nodes, max_depth=8
[12:02:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 100 extra nodes, 0 pr

[12:03:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 202 extra nodes, 0 pruned nodes, max_depth=14
[12:03:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 224 extra nodes, 0 pruned nodes, max_depth=15
[12:03:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 202 extra nodes, 0 pruned nodes, max_depth=15
[12:03:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 200 extra nodes, 0 pruned nodes, max_depth=15
[12:03:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 220 extra nodes, 0 pruned nodes, max_depth=15
[12:03:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 188 extra nodes, 0 pruned nodes, max_depth=15
[12:03:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 220 extra nodes, 0 pruned nodes, max_depth=14
[12:03:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 222 extra nodes, 0 pruned nodes, max_depth=13
[12:03:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 212 extra n

[12:03:03] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 682 extra nodes, 0 pruned nodes, max_depth=15
[12:03:03] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 652 extra nodes, 0 pruned nodes, max_depth=15
[12:03:03] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 602 extra nodes, 0 pruned nodes, max_depth=15
[12:03:03] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 748 extra nodes, 0 pruned nodes, max_depth=15
[12:03:03] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 656 extra nodes, 0 pruned nodes, max_depth=15
[12:03:03] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 664 extra nodes, 0 pruned nodes, max_depth=15
[12:03:03] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 658 extra nodes, 0 pruned nodes, max_depth=15
[12:03:03] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 712 extra nodes, 0 pruned nodes, max_depth=15
[12:03:03] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 710 extra n

...

[12:04:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5588 extra nodes, 0 pruned nodes, max_depth=15
[12:04:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3594 extra nodes, 0 pruned nodes, max_depth=15
[12:04:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1300 extra nodes, 0 pruned nodes, max_depth=15
[12:04:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5670 extra nodes, 0 pruned nodes, max_depth=15
[12:04:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5632 extra nodes, 0 pruned nodes, max_depth=15
[12:04:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2460 extra nodes, 0 pruned nodes, max_depth=15
[12:04:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4110 extra nodes, 0 pruned nodes, max_depth=15
[12:04:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5856 extra nodes, 0 pruned nodes, max_depth=15
[12:04:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 493

[12:04:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3294 extra nodes, 0 pruned nodes, max_depth=15
[12:04:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2788 extra nodes, 0 pruned nodes, max_depth=15
[12:04:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4342 extra nodes, 0 pruned nodes, max_depth=15
[12:04:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3484 extra nodes, 0 pruned nodes, max_depth=15
[12:04:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 5190 extra nodes, 0 pruned nodes, max_depth=15
[12:04:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1902 extra nodes, 0 pruned nodes, max_depth=15
[12:04:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3142 extra nodes, 0 pruned nodes, max_depth=15
[12:04:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1396 extra nodes, 0 pruned nodes, max_depth=15
[12:04:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 348

[12:04:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4814 extra nodes, 0 pruned nodes, max_depth=15
[12:04:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4806 extra nodes, 0 pruned nodes, max_depth=15
[12:04:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2554 extra nodes, 0 pruned nodes, max_depth=15
[12:04:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2394 extra nodes, 0 pruned nodes, max_depth=15
[12:04:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2996 extra nodes, 0 pruned nodes, max_depth=15
[12:04:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3190 extra nodes, 0 pruned nodes, max_depth=15
[12:04:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2184 extra nodes, 0 pruned nodes, max_depth=15
[12:04:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4584 extra nodes, 0 pruned nodes, max_depth=15
[12:04:19] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 223

[12:04:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2898 extra nodes, 0 pruned nodes, max_depth=15
[12:04:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1206 extra nodes, 0 pruned nodes, max_depth=15
[12:04:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 6280 extra nodes, 0 pruned nodes, max_depth=15
[12:04:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2868 extra nodes, 0 pruned nodes, max_depth=15
[12:04:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 2724 extra nodes, 0 pruned nodes, max_depth=15
[12:04:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3624 extra nodes, 0 pruned nodes, max_depth=15
[12:04:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 3364 extra nodes, 0 pruned nodes, max_depth=15
[12:04:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 4240 extra nodes, 0 pruned nodes, max_depth=15
[12:04:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 342

[12:04:45] ======== Monitor: Learner ========
[12:04:45] Configure: 0.000554s, 1 calls @ 554us

[12:04:45] EvalOneIter: 0.000211s, 50 calls @ 211us

[12:04:45] GetGradient: 0.053405s, 50 calls @ 53405us

[12:04:45] PredictRaw: 1.26805s, 50 calls @ 1268048us

[12:04:45] UpdateOneIter: 87.514s, 50 calls @ 87513971us

[12:04:45] ======== Monitor: GBTree ========
[12:04:45] BoostNewTrees: 86.1651s, 50 calls @ 86165126us

[12:04:45] CommitModel: 0.000115s, 50 calls @ 115us

[12:04:45] ======== Monitor: TreePruner ========
[12:04:45] PrunerUpdate: 0.419186s, 50 calls @ 419186us

[12:04:45] ======== Monitor: Learner ========
[12:04:45] Configure: 0.001132s, 1 calls @ 1132us

[12:04:45] ======== Monitor: GBTree ========
[12:04:45] ======== Monitor: TreePruner ========


In [43]:
diamonds_test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.32,Ideal,I,SI1,60.5,58.0,4.43,4.49,2.70
1,1,1.24,Premium,I,SI1,62.9,60.0,6.80,6.74,4.26
2,2,1.66,Premium,D,SI1,62.0,59.0,7.55,7.60,4.70
3,3,0.75,Premium,D,SI2,60.6,56.0,5.94,5.90,3.59
4,4,1.50,Fair,E,SI2,64.8,55.0,7.26,7.15,4.67


In [44]:
diamonds_test.clarity = diamonds_test.clarity.map(dic_clarity)

In [45]:
diamonds_test.color = diamonds_test.color.map(dic_color)

In [46]:
diamonds_test.cut = diamonds_test.cut.map(dic_cut)

In [50]:
diamonds_test['cut'] = lbl.fit_transform(diamonds_test['cut'].astype(str))
diamonds_test['color'] = lbl.fit_transform(diamonds_test['color'].astype(str))
diamonds_test['clarity'] = lbl.fit_transform(diamonds_test['clarity'].astype(str))

In [51]:
diamonds_test_dmatrix = xg.DMatrix(data=diamonds_test, enable_categorical=True)

In [52]:
y_price_pred = xgb_reg.predict(diamonds_test_dmatrix)

In [53]:
y_price_pred

array([6.057235 , 8.494676 , 9.387988 , ..., 6.4087543, 8.706489 ,
       7.9942145], dtype=float32)

In [54]:
diamonds_test['price'] = y_price_pred

In [55]:
diamonds_test.drop(columns=['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y',
       'z'], inplace=True)

In [56]:
diamonds_test

,id,price
0,0,6.057235
1,1,8.494676
2,2,9.387988
3,3,7.822424
4,4,8.915586
...,...,...
13480,13480,8.556455
13481,13481,8.404564
13482,13482,6.408754
13483,13483,8.706489


In [57]:
diamonds_test.to_csv('diamonds_xgb.csv', index = False)